In [60]:
import numpy as np
import trimesh
import tqdm 

In [61]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    p0 = vertices[faces[:, 0]]
    p1 = vertices[faces[:, 1]]
    p2 = vertices[faces[:, 2]]

    area = np.linalg.norm(np.cross(p1 - p0, p2 - p0), axis=1) / 2
    prob = area / np.sum(area)
    sample_faces = np.random.choice(faces.shape[0], size=sample_num, p=prob)

    p0 = vertices[faces[sample_faces, 0]]
    p1 = vertices[faces[sample_faces, 1]]
    p2 = vertices[faces[sample_faces, 2]]
    v1 = p1 - p0
    v2 = p2 - p0

    u = np.random.rand(sample_num).reshape(-1, 1)
    v = np.random.rand(sample_num).reshape(-1, 1)
    uniform_pc = p0 + np.where(u + v < 1, u * v1 + v * v2, (1 - u) * v1 + (1 - v) * v2)

    # -------- TODO -----------
    return area, prob, uniform_pc
        

In [62]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    indices = []
    indices.append(np.random.randint(pc.shape[0]))
    dist = np.linalg.norm(pc - pc[indices[0]], axis=-1)

    for _ in tqdm.tqdm(range(sample_num - 1)):
        farthest = np.argmax(dist)
        indices.append(farthest)
        new_dist = np.linalg.norm(pc - pc[farthest], axis=-1)
        dist = np.minimum(dist, new_dist)
    results = pc[indices]
    # -------- TODO -----------

    return results

In [63]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [ ]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

100%|██████████| 511/511 [00:00<00:00, 6420.15it/s]


<Figure size 640x480 with 0 Axes>

In [ ]:
# task 3: metrics

from earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
CD = []
EMD = []
for _ in range(5):
    _, _, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)
    _, _, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

    dist = np.linalg.norm(uniform_pc[:, None, :] - fps_pc[None, :, :], axis=-1)
    cd = np.mean(np.min(dist, axis=0) + np.min(dist, axis=1))
    CD.append(cd)

    uniform_pc = [tuple(p) for p in uniform_pc]
    fps_pc = [tuple(p) for p in fps_pc]
    EMD.append(earthmover_distance(uniform_pc, fps_pc))

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)
# -----------TODO---------------

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

100%|██████████| 511/511 [00:00<00:00, 10382.59it/s]


move 0.001953125 dirt from (18.672175100916625, 49.69070957629086, 39.37221132103379) to (18.02687750388757, 51.13182992372795, 37.91625338749189) for a cost of 0.004194924379729869
move 0.001953125 dirt from (34.16294010046549, 28.039340604912613, 41.79027409160731) to (32.663544251995944, 27.285528958652908, 42.10915352867669) for a cost of 0.0033364177442610967
move 0.001953125 dirt from (41.205415284436285, 14.181233164180323, 36.03366418219955) to (40.64846876858025, 14.0790670314506, 36.16581020445269) for a cost of 0.0011356542874251224
move 0.001953125 dirt from (15.372018171855828, 30.534136226167142, 32.19488479818694) to (15.061719538586152, 30.48233187190457, 31.373700198174213) for a cost of 0.0017175433888599388
move 0.001953125 dirt from (21.68702556404094, 33.10524481968505, 26.41048330597521) to (19.30395682008743, 32.63157685368549, 26.28677758847553) for a cost of 0.004751628831350388
move 0.001953125 dirt from (40.54426359964576, 13.404930644429813, 28.4427776661887

100%|██████████| 511/511 [00:00<00:00, 10042.12it/s]


move 0.001953125 dirt from (29.64224646523711, 8.487611292265482, 23.342624724635204) to (28.415368847785036, 7.419186077809085, 24.651257829084013) for a cost of 0.004077908789107363
move 0.001953125 dirt from (23.712549080377713, 24.552391601390607, 40.77012361432057) to (26.684123571487046, 24.599517900307205, 42.156355614174196) for a cost of 0.006404974089808906
move 0.001953125 dirt from (26.308036435155522, 7.877031457956951, 40.46226618721864) to (24.473565606269283, 11.181903010366547, 41.95599868005117) for a cost of 0.00793812473514376
move 0.001953125 dirt from (18.02747984919191, 52.76490510582583, 28.73528058210994) to (19.14747067593804, 53.04032117476137, 35.2274230968052) for a cost of 0.012878508138996688
move 0.001953125 dirt from (38.7016858642358, 29.36207267110905, 22.321089167455757) to (42.85508020718285, 26.6017655740923, 21.367244129628784) for a cost of 0.009916760222401864
move 0.001953125 dirt from (29.05552164594667, 37.43670283537361, 28.472388598007246) 

100%|██████████| 511/511 [00:00<00:00, 11365.59it/s]


move 0.001953125 dirt from (44.46863192110163, 14.008421063774803, 28.847550926048772) to (45.04084189614491, 14.012724690250824, 29.377000174866126) for a cost of 0.0015226351099722401
move 0.001953125 dirt from (44.16388378797243, 14.29150386923488, 33.407455074662685) to (46.1081018632705, 13.050582847522184, 33.51001507415667) for a cost of 0.004509303102271621
move 0.001953125 dirt from (24.30164643140052, 20.806213867481148, 41.92300576043516) to (23.41194770555918, 18.872688525149755, 41.525526495891484) for a cost of 0.004228898491410219
move 0.001953125 dirt from (20.657172069918403, 21.960335778736603, 35.37855243043498) to (20.10314769321071, 23.04883307310671, 35.04583975769596) for a cost of 0.0024724333834866665
move 0.001953125 dirt from (17.96710563423128, 46.84242887077194, 19.021639056125238) to (17.150834140426042, 46.34262378406066, 20.14153207560341) for a cost of 0.0028773082266750852
move 0.001953125 dirt from (52.31060094302011, 16.550328259041624, 25.6443600557

100%|██████████| 511/511 [00:00<00:00, 9000.54it/s]


move 0.001953125 dirt from (52.17663809102167, 10.162763862088967, 24.92721689511107) to (51.02560602488358, 8.51054775281575, 24.558130628316825) for a cost of 0.003998384894149783
move 0.001953125 dirt from (37.72216206025566, 28.448021687607635, 21.723567001081538) to (37.057546315679694, 29.593414557492512, 22.281310585636927) for a cost of 0.0028064692425180462
move 0.001953125 dirt from (16.781859606325416, 42.61106587509676, 23.042348293235644) to (15.884202317286654, 43.50809207310143, 23.19079931097592) for a cost of 0.0024954810790934626
move 0.001953125 dirt from (12.794675696338576, 48.53174463927375, 25.860699175278885) to (12.278049242968153, 46.78291459821744, 24.779318120866744) for a cost of 0.004140760606226068
move 0.001953125 dirt from (39.65757721018988, 28.45659355063429, 21.87430940316749) to (36.6547206139245, 26.21905020338007, 20.81173467520128) for a cost of 0.007602854358599369
move 0.001953125 dirt from (25.69827812598132, 11.488227605106411, 20.70913792888

100%|██████████| 511/511 [00:00<00:00, 9175.75it/s]


move 0.001953125 dirt from (44.27659051112262, 34.041989052415374, 27.711528884909985) to (42.829746353439276, 34.58508949777096, 29.231616956407464) for a cost of 0.004233816347540753
move 0.001953125 dirt from (27.650586083926388, 31.645282683955397, 22.997134408789755) to (28.343626896803954, 29.618218734176676, 21.784324659555484) for a cost of 0.004808100807645455
move 0.001953125 dirt from (41.17943979653135, 14.191617454806856, 36.13208086031569) to (40.61706472356024, 13.524946540598078, 35.09808298923382) for a cost of 0.002642042822111098
move 0.001953125 dirt from (19.320011126218564, 25.320013012689596, 29.544974705249967) to (18.957625978067735, 26.298744071514598, 30.76574549035352) for a cost of 0.0031368907022982576
move 0.001953125 dirt from (50.355484158054715, 24.827847522755732, 39.73336254351874) to (51.90771798645628, 23.097079152630627, 38.1281241201468) for a cost of 0.005517977172008394
move 0.001953125 dirt from (17.496806925430025, 48.43911677305386, 19.05034